In [1]:
import numpy as np

In [2]:
import scipy.stats as stats

In [3]:
from GPy.inference.latent_function_inference import EPDTC

In [4]:
raw_data = np.load('binary_class_ds.npz')
X_crabs_train = raw_data['X_crabs_train']
Y_crabs_train = raw_data['Y_crabs_train']
X_liver_train = raw_data['X_liver_train']
Y_liver_train = raw_data['Y_liver_train']
X_diabetes_train = raw_data['X_diabetes_train']
Y_diabetes_train = raw_data['Y_diabetes_train']
X_banana_train = raw_data['X_banana_train']
Y_banana_train = raw_data['Y_banana_train']
X_iono_train = raw_data['X_iono_train']
Y_iono_train = raw_data['Y_iono_train']
X_sonar_train = raw_data['X_sonar_train']
Y_sonar_train = raw_data['Y_sonar_train']
X_cancer_train = raw_data['X_cancer_train']
Y_cancer_train = raw_data['Y_cancer_train']
X_heart_train = raw_data['X_heart_train']
Y_heart_train = raw_data['Y_heart_train']
X_ringnorm_train = raw_data['X_ringnorm_train']
Y_ringnorm_train = raw_data['Y_ringnorm_train']
X_twonorm_train = raw_data['X_twonorm_train']
Y_twonorm_train = raw_data['Y_twonorm_train']

In [5]:
import GPy

In [6]:
print Y_cancer_train.shape
Y_cancer_train= Y_cancer_train.reshape(-1,1)

(569,)


In [7]:
# X_train = X_banana_train
# Y_train = Y_banana_train
X_train = X_cancer_train
Y_train = Y_cancer_train

In [8]:
# kern_f = GPy.kern.RBF(X_train.shape[1], variance=4, lengthscale=(X_train.max(axis=0) - X_train.min(axis=0))/100.0, ARD=True, name='f_rbf') 
kern_f = GPy.kern.RBF(X_train.shape[1], variance=10, lengthscale=1.0, ARD=True, name='f_rbf') 
kern_f += GPy.kern.Bias(X_train.shape[1], variance=0.1, name='f_bias')
kern_f += GPy.kern.White(X_train.shape[1], variance=1e-5, name='f_white')

lik = GPy.likelihoods.Bernoulli()
kern_f.name = 'f_kern'

num_inducing = int(X_train.shape[0]*0.10)
Z = np.random.permutation(X_train.copy())[:num_inducing, :]
m_svgp1 = GPy.core.SVGP(X=X_train.copy(), Y=Y_train.copy(), Z=Z.copy(), kernel=kern_f.copy(), likelihood=lik.copy(), mean_function=None, Y_metadata=None, name='svgp_crabs')

In [9]:
print(m_svgp1)


Name : svgp_crabs
Objective : 1755.91085772
Number of Parameters : 3365
Number of Optimization Parameters : 3365
Updates : True
Parameters:
  svgp_crabs.               |      value  |  constraints  |  priors
  inducing_inputs           |   (56, 30)  |               |        
  f_kern.f_rbf.variance     |       10.0  |      +ve      |        
  f_kern.f_rbf.lengthscale  |      (30,)  |      +ve      |        
  f_kern.f_bias.variance    |        0.1  |      +ve      |        
  f_kern.f_white.variance   |      1e-05  |      +ve      |        
  q_u_chol                  |  (1596, 1)  |               |        
  q_u_mean                  |    (56, 1)  |               |        


In [10]:
m_svgp1.optimize('lbfgs', max_iters=1400)

In [11]:
print(m_svgp1)
# print(m_svgp1['f_kern.f_rbf.lengthscale'])


Name : svgp_crabs
Objective : 462.475091024
Number of Parameters : 3365
Number of Optimization Parameters : 3365
Updates : True
Parameters:
  svgp_crabs.               |             value  |  constraints  |  priors
  inducing_inputs           |          (56, 30)  |               |        
  f_kern.f_rbf.variance     |    0.298801078556  |      +ve      |        
  f_kern.f_rbf.lengthscale  |             (30,)  |      +ve      |        
  f_kern.f_bias.variance    |   0.0830643299478  |      +ve      |        
  f_kern.f_white.variance   |  9.9984549431e-06  |      +ve      |        
  q_u_chol                  |         (1596, 1)  |               |        
  q_u_mean                  |           (56, 1)  |               |        


In [12]:
def get_svgp_model(X_train, Y_train, Z=None, percent=0.15):
    kern_f = GPy.kern.RBF(X_train.shape[1], variance=10.0, lengthscale=1.0, ARD=True, name='f_rbf') 
#     kern_f = GPy.kern.RBF(X_train.shape[1], variance=10.0, lengthscale=(X_train.max(axis=0) - X_train.min(axis=0))/6.0, ARD=True, name='f_rbf') 
    kern_f += GPy.kern.Bias(X_train.shape[1], variance=0.1, name='f_bias')
    kern_f += GPy.kern.White(X_train.shape[1], variance=1e-5, name='f_white')

    lik = GPy.likelihoods.Bernoulli()
    kern_f.name = 'f_kern'
    num_inducing = int(X_train.shape[0]*percent)
    print "percentage of inducing points:", percent
    Z = np.random.permutation(X_train.copy())[:num_inducing, :]
    m_svgp = GPy.core.SVGP(X=X_train.copy(), Y=Y_train.copy(), Z=Z.copy(), kernel=kern_f.copy(), likelihood=lik.copy(), mean_function=None, Y_metadata=None, name='svgp_crabs')
    return m_svgp  
    

In [13]:
def get_epdtc_model(X_train, Y_train, Z=None, percent=0.15):
    kern_f = GPy.kern.RBF(X_train.shape[1], variance=10.0, lengthscale=1.0, ARD=True, name='f_rbf') 
    kern_f += GPy.kern.Bias(X_train.shape[1], variance=0.1, name='f_bias')
    kern_f += GPy.kern.White(X_train.shape[1], variance=1e-5, name='f_white')
    
    epdtc = GPy.inference.latent_function_inference.EPDTC(ep_mode='alternated',  parallel_updates=True)
    lik = GPy.likelihoods.Bernoulli()
    kern_f.name = 'f_kern'
    num_inducing = int(X_train.shape[0]*percent)
    print "percentage of inducing points:", percent
    Z = np.random.permutation(X_train.copy())[:num_inducing, :]
#     m_epdtc = GPy.core.SparseGP(X=X_train.copy(), Y=Y_train.copy(), Z=Z.copy(), kernel=kern_f.copy(), likelihood=lik.copy(), inference_method=epdtc, mean_function=None, Y_metadata=None, name='epdtc')
    m_epdtc = GPy.core.SparseGPClassification(X=X_train.copy(), Y=Y_train.copy(), Z=Z.copy(), kernel=kern_f.copy(), likelihood=lik.copy(), inference_method=epdtc, mean_function=None, Y_metadata=None, name='epdtc')
    return m_epdtc
    

In [14]:
import GPy.util.classification


class Dataset():
    def __init__(self, X, Y, Xtest=None, Ytest=None, K=10, seed=42):
        X_orig = X.copy()
        Y_orig = Y.copy()
        np.random.seed(seed=seed)
        if Y.ndim == 1:
            Y = Y.reshape(-1,1)
        
        randomize = np.arange(X.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize,:]
        Y = Y[randomize,:]
        self.K = K
        self.X = X
        self.Y = Y
        self.m_list= []
        self.X_test_list = []
        self.Y_test_list = []
        self.partition_flag = False
        self.model_fn_flag = False
    
    def split_data(self):
        X_train_list = []
        Y_train_list = []
        num_batch = self.X.shape[0]/self.K
        rem = self.X.shape[0] % self.K
        

        for i in range(self.K-1):
            X_batch = self.X[i*num_batch:(i+1)*num_batch,:]  
            Y_batch = self.Y[i*num_batch:(i+1)*num_batch,:]
            X_train_list.append(X_batch)
            Y_train_list.append(Y_batch)
        X_train_list.append(self.X[(self.K-1)*num_batch:self.K*num_batch+rem,:])
        Y_train_list.append(self.Y[(self.K-1)*num_batch:self.K*num_batch+rem,:])
        self.X_train_list = X_train_list
        self.Y_train_list = Y_train_list
        self.partition_flag = True
        
    def show_data_partition(self):
        if self.partition_flag is False:
            self.split_data()
        for i in self.X_train_list:
            print len(i)
    
    def set_model_function(self, fn, **kwargs):
        self.get_model_fn = fn
        self.fn_args = dict()
        self.fn_kwargs = kwargs
        self.model_fn_flag = True
    
    def get_model_function(self):
        return self.get_model_fn

    def get_model_list(self):
        return self.m_list
    
    def get_test_set(self, i):
        return self.X_test_list[i], self.Y_test_list[i]
    
    def train_KFold(self):
        if self.partition_flag is False:
            self.split_data()
        
        if self.model_fn_flag is False:
            self.set_model_function()
        
        for i in range(self.K):
            Xtest = self.X_train_list.pop(i)
            Ytest =  self.Y_train_list.pop(i)
            Xtrain_all = np.concatenate(self.X_train_list, axis=0)
            Ytrain_all = np.concatenate(self.Y_train_list, axis=0)
            self.X_train_list.insert(i, Xtest)
            self.Y_train_list.insert(i, Ytest)
            self.X_test_list.append(Xtest)
            self.Y_test_list.append(Ytest)
#             m = get_svgp_model(Xtrain_all, Ytrain_all)
            m = self.get_model_fn(Xtrain_all, Ytrain_all, **self.fn_kwargs)
            m.optimize('lbfgs', max_iters=900)
            self.m_list.append(m)



class EvalClassification():
    def __init__(self, m):
        self.m = m
    
    def NLL(self):
        return self.m.log_likelihood()
    
    def TestNLL1(self, Xtest, Ytest):
        mu = self.m._raw_predict(Xtest)
        predict_mean = self.m.likelihood.gp_link.transf(mu)
        target = Ytest
        probs, probs_var = self.m.predict(Xtest)
        nlprobs = np.log(probs)
        nlprobsvar = np.log(probs_var)
        NLP = np.mean(np.log(probs), axis=0)
#         NLP_Var = np.median(np.log(nlprobsvar), axis=0)
        self.probs = probs
        return probs, NLP
    
    def TestNLL(self, Xtest, Ytest, median=False):
        mu, var = self.m._raw_predict(Xtest)
        nll = -stats.norm.logcdf(1.0 * Ytest * mu / np.sqrt(1 + var))
        if median:
            val= np.median(nll)
        else:
            val= np.mean(nll)
        sigma2 = 2*np.nanstd(nll)
        return val, sigma2
        
    
    def conf_matrix(self, Xtest, Ytest, threshold=0.5):
        assert self.probs.size == Ytest.size
        decision = np.zeros((Ytest.size, 1))
        decision[self.probs>threshold] = 1
        diff = decision - Ytest
        f_0 = diff[diff==-1].size
        f_1 = diff[diff==1].size    
        t_1 = np.sum(decision[diff ==0])
        t_0 = Ytest.size - f_0 - f_1 - t_1
        correct = t_0+t_1
        false = Ytest.size -1
        error = (f_0 + f_1)/ np.float(Ytest.size)
        return error, f_0, f_1, t_0, t_1
        

#### This is just very easy to use. The above classes define the training and validation procedure with K-Fold Cross validation. I can just use any function which returns me a sparse GP model(either SVGP or EPDTC), and then I can just train and evaluate with help of the classes defined above.

In [15]:
ds1 = Dataset(X=X_train, Y=Y_train, K=5)
ds1.split_data()
ds1.show_data_partition()
ds1.set_model_function(get_epdtc_model, percent=0.16)
ds1.train_KFold()

113
113
113
113
117
percentage of inducing points: 0.16


AttributeError: 'module' object has no attribute 'SparseGPClassification'

In [16]:
ds1 = Dataset(X=X_train, Y=Y_train, K=10)
ds1.split_data()
ds1.show_data_partition()
ds1.set_model_function(get_svgp_model, percent=0.12)
ds1.train_KFold()

56
56
56
56
56
56
56
56
56
65
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12
percentage of inducing points: 0.12


In [17]:
test_nll_list = []
test_nll_list2 = []
error_list= []
sigma2_list = []

for ind, mi in enumerate(ds1.get_model_list()):
    ec = EvalClassification(mi)
    xtest, ytest = ds1.get_test_set(ind)
    testnll1, sigma2 = ec.TestNLL(xtest, ytest)
    
    probs2, testnll2  = ec.TestNLL1(xtest, ytest)
#     testnll2 = ec.TestNLL1(xtest, ytest)
    error, f_0, f_1, t_0, t_1 = ec.conf_matrix(xtest, ytest)
    test_nll_list.append(testnll1)
    sigma2_list.append(sigma2)
    error_list.append(error)

print np.median(test_nll_list)
print np.median(sigma2_list)
print np.mean(error_list)
print test_nll_list


0.553660201543
0.206344864837
0.372637362637
[0.54858110072053112, 0.56271607478895114, 0.55294604154202942, 0.54938564838796633, 0.55102422681373353, 0.55648216565882869, 0.54919883862715024, 0.56147910054350336, 0.63325760332040382, 0.55437436154349351]
